In [1]:
import numpy as np
import geopandas as gpd
import xarray as xr
import shapely
import random
from hextraj import HexProj

/home/fkirch/anaconda3/lib/python3.11/site-packages/hextraj/redblobhex_array.py:8: RuntimeWarning: invalid value encountered in cast
  INTNaN = np.array(np.nan).astype(int)[()]


In [2]:
ds_traj = xr.open_dataset("nwshelf.nc")

In [3]:
lat_range = [46.0, 64.0]
lon_range = [-15, 10]
n_hex = 10
hex_height = 2**0.5
hex_size_meters = np.diff(lat_range) * 111e3 / n_hex / hex_height

In [4]:
hex_proj = HexProj(
    lon_origin=np.mean(lon_range),
    lat_origin=np.mean(lat_range),
    hex_size_meters=hex_size_meters
)

/home/fkirch/anaconda3/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [5]:
hex_labels = xr.apply_ufunc(
    hex_proj.lon_lat_to_hex_AoS,
    ds_traj.lon, 
    ds_traj.lat,
    dask="parallelized",
    output_dtypes=[tuple, ],
).rename("hex_labels")

/home/fkirch/anaconda3/lib/python3.11/site-packages/hextraj/redblobhex_array.py:98: RuntimeWarning: invalid value encountered in cast
  qi = np.round_(hex.q).astype(int)
/home/fkirch/anaconda3/lib/python3.11/site-packages/hextraj/redblobhex_array.py:99: RuntimeWarning: invalid value encountered in cast
  ri = np.round_(hex.r).astype(int)
/home/fkirch/anaconda3/lib/python3.11/site-packages/hextraj/redblobhex_array.py:100: RuntimeWarning: invalid value encountered in cast
  si = np.round_(hex.s).astype(int)


In [6]:
unique_hex_labels = np.unique(hex_labels)
corners = hex_proj.hex_corners_lon_lat_AoS(unique_hex_labels)

In [7]:
hex_polygons = [
    shapely.geometry.Polygon(corners) for corners in np.array(corners).T[1:]
]
hex_lons, hex_lats = np.array(
    [hex_proj.hex_to_lon_lat_SoA(hex) for hex in unique_hex_labels[1:]]
).T.squeeze().astype(float)
ids = np.arange(len(hex_polygons)).astype(int)
depth = np.random.randint(10, 1000, len(hex_polygons)).astype(int)
disease = np.array(random.choices([0, 1], k=len(hex_polygons))).astype(int)
rest = np.array(random.choices([0, 1], k=len(hex_polygons))).astype(int)
substrate = np.array(random.choices(["A", "B", "C"], k=len(hex_polygons))).astype(str)

In [8]:
def create_connections(ids, n_connections):
    to_IDs = np.array(random.sample(list(ids), k=np.random.randint(1, int(n_connections))))
    weights = np.random.uniform(size=len(to_IDs))
    weights = weights / sum(weights)
    return dict(zip(to_IDs.astype(str), weights.astype(float)))
    # return to_IDs.astype(int), weights.astype(float)

# connectivity = []
# for i in range(len(hex_polygons)):
#     to_IDs, weights = create_connections(ids, len(hex_polygons))
#     connectivity.append({"toID": to_IDs.tolist(), "weight": weights.tolist()})

connectivity = [create_connections(ids, len(hex_polygons)/3) for i in range(len(hex_polygons))]

In [9]:
features = gpd.GeoDataFrame(
    data={
        "id": ids,
        "lon": hex_lons,
        "lat": hex_lats,
        "depth": depth,
        "disease": disease,
        "rest": rest,
        "substrate": substrate,
        "connectivity": connectivity
    },
    geometry=hex_polygons
)

features.to_file('hex_features.geojson', driver='GeoJSON')